In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
df = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")

In [4]:
df = df.copy()
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           374 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [5]:
# Change objects into categorical numbers
from sklearn import preprocessing

# Columns we want to transform
categorical = ["Gender", "Occupation", "BMI Category", "Sleep Disorder"]

label_encoder = preprocessing.LabelEncoder()

for label in categorical:
    df[label] = label_encoder.fit_transform(df[label])

df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,1,27,9,6.1,6,42,6,3,126/83,77,4200,1
1,2,1,28,1,6.2,6,60,8,0,125/80,75,10000,1
2,3,1,28,1,6.2,6,60,8,0,125/80,75,10000,1
3,4,1,28,6,5.9,4,30,8,2,140/90,85,3000,2
4,5,1,28,6,5.9,4,30,8,2,140/90,85,3000,2


In [6]:
# Now we drop our old blood pressure column
df.drop("Blood Pressure", axis=1, inplace=True)

In [7]:
# View our dataframe
df.head(20)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder
0,1,1,27,9,6.1,6,42,6,3,77,4200,1
1,2,1,28,1,6.2,6,60,8,0,75,10000,1
2,3,1,28,1,6.2,6,60,8,0,75,10000,1
3,4,1,28,6,5.9,4,30,8,2,85,3000,2
4,5,1,28,6,5.9,4,30,8,2,85,3000,2
5,6,1,28,9,5.9,4,30,8,2,85,3000,0
6,7,1,29,10,6.3,6,40,7,2,82,3500,0
7,8,1,29,1,7.8,7,75,6,0,70,8000,1
8,9,1,29,1,7.8,7,75,6,0,70,8000,1
9,10,1,29,1,7.8,7,75,6,0,70,8000,1


In [8]:
# Testing for duplicates excluding the ID column
columns = ['Gender', 'Age', 'Occupation', 'Sleep Duration',
       'Quality of Sleep', 'Physical Activity Level', 'Stress Level',
       'BMI Category', 'Heart Rate', 'Daily Steps', 'Sleep Disorder']

df.duplicated(subset=columns).value_counts()

True     242
False    132
Name: count, dtype: int64

In [9]:
# Removing duplicates
duplicates = df.duplicated(subset=columns)

for i, value in enumerate(duplicates):
    if value==True:
        df.drop(index=i, inplace=True)

In [10]:
# Now we check again
df.duplicated(subset=columns).value_counts()

False    132
Name: count, dtype: int64

In [11]:
# Finally we remove our PersonID column, as it doesn't add much value to the dataset
df.drop("Person ID", axis=1, inplace=True)

In [12]:
# Imports from sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
# Models
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
# Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

In [34]:
df.drop("Occupation", axis=1, inplace=True)
df.drop("BMI Category", axis=1, inplace=True)
df.drop("Heart Rate", axis=1, inplace=True)
df.drop("Daily Steps", axis=1, inplace=True)
df.drop("Sleep Disorder", axis=1, inplace=True)

KeyError: "['Occupation'] not found in axis"

In [14]:
df.head()

,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level
0,1,27,6.1,6,42,6
1,1,28,6.2,6,60,8
3,1,28,5.9,4,30,8
5,1,28,5.9,4,30,8
6,1,29,6.3,6,40,7


In [35]:
df.drop("Gender", axis=1, inplace=True)

In [36]:
df.head()

,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level
0,27,6.1,6,42,6
1,28,6.2,6,60,8
3,28,5.9,4,30,8
5,28,5.9,4,30,8
6,29,6.3,6,40,7


In [37]:
# Split the data
X = df.drop("Quality of Sleep", axis=1)
y = df["Quality of Sleep"]

In [38]:
# Split into train and test sets
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [39]:
X_test.head()

,Age,Sleep Duration,Physical Activity Level,Stress Level
144,38,7.1,60,5
219,43,6.5,45,7
34,31,7.7,75,6
70,33,6.1,30,8
191,43,6.4,45,7


In [40]:
grid_rf = {"n_estimators": np.arange(100, 2000, 20),
          "max_features": ['sqrt', 'log2', 0.5],
          "max_depth": np.arange(10, 110, 11),
          "min_samples_split": [2, 5, 10],
          "min_samples_leaf": [1, 2, 4],
          "bootstrap": [True, False]}

In [41]:
# The same for our RandomForestRegressor
rs_rf = RandomizedSearchCV(RandomForestRegressor(),
                          param_distributions=grid_rf,
                          n_iter=20,
                          cv=5,
                          random_state=42)
rs_rf.fit(X_train, y_train);

In [42]:
# Function to evaluate all models
def models_evaluate(models, X_test, y_test):
    collection = {}
    for name, model in models.items():
        model_scoring(name, model, X_test, y_test, collection)
    return collection

In [28]:
# Create dictionary for our selected models and evaluate them with our function
refined_models = {"RandomForest": rs_rf}

rs_results = models_evaluate(refined_models, X_test, y_test)

NameError: name 'model_scoring' is not defined

In [43]:
y_pred = np.array(rs_rf.predict(X_test))
rounded_arr = np.round(y_pred)

In [44]:
print(rounded_arr)

[8. 6. 7. 6. 6. 9. 8. 9. 8. 7. 8. 6. 5. 5. 6. 6. 8. 8. 8. 8. 6. 5. 8. 7.
 9. 8. 9.]


In [45]:
import pickle
with open('rf.pkl', 'wb') as f:
    pickle.dump(rs_rf, f)

In [46]:
with open('rf.pkl', 'rb') as f:
    rs_rf_model = pickle.load(f)

In [47]:
predictions =np.array(rs_rf_model.predict(X_test))
print(np.round(predictions))

[8. 6. 7. 6. 6. 9. 8. 9. 8. 7. 8. 6. 5. 5. 6. 6. 8. 8. 8. 8. 6. 5. 8. 7.
 9. 8. 9.]
